In [146]:
from misc import NDCG
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
data1 = pd.read_csv("Data/Table_S6_GDSC_Drug_response_IC50.csv")
data1.drop(['Unnamed: 0'], axis='columns', inplace=True)

scaler = MinMaxScaler()
scaler.fit(data1)

n_fold = 3
NDCG_List = [1,2,5,10,20,50]
NDCG_201_fold = []

for cv in range(n_fold):
    Name_npz = "Result/{}_{}.npz".format("KRR_CV",cv)
    CV = np.load(Name_npz)
    CV_test = CV['Y_true']
    CV_pred = CV["Y_pred"]
    
    Y_test = CV_test
    Y_test = scaler.inverse_transform(Y_test)
    NDCG_201 = []
    for i in NDCG_List:
        ndcg_201 = NDCG(CV_test, CV_pred, i)
        NDCG_201.append(ndcg_201.sum()/ndcg_201.shape[0])
        
    NDCG_201_fold.append(NDCG_201)
    
NDCG_Sum = np.zeros([1,len(NDCG_List)])
NDCG_201_fold = np.array(NDCG_201_fold)
for cv in range(n_fold):
    NDCG_Sum += NDCG_201_fold[cv,:]
    
Result_NDCG = pd.DataFrame(NDCG_Sum.reshape(len(NDCG_List),1)/n_fold, columns =["NDCG"], index=NDCG_List)

RMSE_fold = []
PCC_fold = []
SCC_fold = []

for cv in range(n_fold):
    RMSE = []
    PCC = []
    SCC = []
    Name_npz = "Result/{}_{}.npz".format("KRR_CV",cv)
    CV = np.load(Name_npz)
    CV_test = CV['Y_true']
    CV_pred = CV["Y_pred"]
    CV_test = scaler.inverse_transform(CV_test)
    CV_pred = scaler.inverse_transform(CV_pred)

    for i in range(data1.shape[1]):
        y_test = CV_test[:, i]
        y_pred = CV_pred[:, i]
        y_pred = y_pred[~np.isnan(y_test)]
        y_test = y_test[~np.isnan(y_test)]

        RMSE.append(mean_squared_error(y_test,y_pred))
        PCC.append(stats.pearsonr(y_test,y_pred)[0])
        SCC.append(stats.spearmanr(y_test,y_pred)[0])
        
    RMSE_fold.append(RMSE)
    PCC_fold.append(PCC)
    SCC_fold.append(SCC)
    
np.array(RMSE_fold).shape
RMSE_Sum = np.zeros([1,data1.shape[1]])
PCC_Sum = np.zeros([1,data1.shape[1]])
SCC_Sum = np.zeros([1,data1.shape[1]])

RMSE_fold = np.array(RMSE_fold)
PCC_fold = np.array(PCC_fold)
SCC_fold = np.array(SCC_fold)

for cv in range(n_fold):
    RMSE_Sum += RMSE_fold[cv,:]
    PCC_Sum += PCC_fold[cv,:]
    SCC_Sum += SCC_fold[cv,:]

Result_RPC = pd.DataFrame(RMSE_Sum.reshape(data1.shape[1],1)/n_fold, columns = ["RMSE"])
Result_RPC["PCC"] = PCC_Sum.reshape(data1.shape[1],1)/n_fold
Result_RPC["SCC"] = SCC_Sum.reshape(data1.shape[1],1)/n_fold

#Result_RPC.to_csv("Result_RPC.csv",mode = 'w')
#Result_NDCG.to_csv("Result_NDCG.csv", mode ='w')


In [147]:

Result_RPC.to_csv("KRR_RPC.csv",mode = 'w')
Result_NDCG.to_csv("KRR_NDCG.csv", mode ='w')


In [148]:
Result_RPC


,RMSE,PCC,SCC
0,1.882637,0.390928,0.356555
1,1.878981,0.396636,0.441663
2,2.909506,0.249269,0.243577
3,3.459131,0.367892,0.365684
4,7.621476,0.351765,0.363409
...,...,...,...
196,2.374561,0.595760,0.462469
197,0.443572,0.619277,0.567644
198,2.689844,0.638202,0.647411
199,2.006340,0.529659,0.512082
